<a href="https://colab.research.google.com/github/SELF-msselve/UTN-DataEngineering/blob/main/CEL_Data_Eng_Almacenamiento_CSV_vs_Parquet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Almacenamiento de datos**

Al momento de almacenar datos, solemos trabajar con formatos particulares que ofrecen ventajas como comprimir el volúmen de los datos, acelerar los tiempos de lectura y escritura, etc.

Uno de los formatos más populares en Data Engineering es el formato parquet. Se trata de un formato de archivo diseñado para un almacenamiento y recuperación eficiente de los datos.

Es posible utilizarlo con diferentes lenguajes y herramientas.

Esta notebook tiene como objetivo mostrar, de forma práctica, las ventajas del formato Parquet, frente a otro formato como el CSV.

Antes de continuar, intenta responder...
¿Cuál es la principal diferencia entre Parquet y CSV? Parquet está orientado a ... y CSV a ...

#### **Dataset de muestra**
Vamos a utilizar un dataset de ejemplo para mostrar los beneficios del formato Parquet.
El dataset será descargado desde una URL con Python y lo guardaremos, tanto:
- en formato CSV.
- como en Parquet.

In [ ]:
import pandas as pd

In [ ]:
# Descarga del dataset, usando el link de github
df = pd.read_csv("https://data.montgomerycountymd.gov/api/views/v76h-r7br/rows.csv")

# Guardar el dataset en formato csv y parquet
df.to_csv("data.csv", index=False)
df.to_parquet("data.parquet", index=False)

#### **Volúmen**
Una de las características del formato parquet, es que comprime los datos, por ende reduce el uso del espacio en disco, lo cual es una gran ventaja cuando se trata de grandes volúmenes de datos. Veamos cuanto volúmen ocupa cada formato

In [ ]:
import os

print(f"El archivo CSV pesa {os.path.getsize('data.csv') / 1024 / 1024 } MB")
print(f"El archivo Parquet pesa {os.path.getsize('data.parquet') / 1024 / 1024 } MB")

El archivo CSV pesa 25.759827613830566 MB
El archivo Parquet pesa 5.420609474182129 MB


Si bien se trata de un dataset pequeño, podemos observar la gran diferencia en cuanto al volúmen. El tamaño del archivo en formato parquet es, aproximadamente, cinco veces menor a la del CSV. Si lo pensamos a gran escala, con datos de muchos GB o TB, podemos apreciar la gran diferencia.

#### **Tiempo de lectura**

A continuación, veremos las diferencias en cuanto a los tiempos de lectura.

In [ ]:
%timeit pd.read_csv("data.csv")

315 ms ± 14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%timeit pd.read_parquet("data.parquet")

179 ms ± 9.33 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


`%timeit` se encarga de ejecutar varias veces la línea de código para recopilar y cálcular métricas.
Así nos muestra, la duración promedio de la ejecución con un desvío standard.

Si ejecutas por tu cuenta puede que los números varíen pero, en definitiva, la lectura de un archivo Parquet es mas rápida que la del CSV. Se puede apreciar las diferencias tanto en el promedio como en el desvío standard.

Si bien, se trata de milisegundos para este caso, dado que los archivos son pequeños, cuando se trata de minutos u horas, se puede notar la gran diferencia.

#### **Tiempo de procesamiento**

Por último, vamos a comparar las tiempos de procesamiento. Los formatos columnares, como el parquet, están diseñados para realizar operaciones analíticas sobre los datos, que implique aplicar cálculos sobre toda columna

In [ ]:
# Cargar los datasets
df_csv = pd.read_csv("data.csv")
df_parquet = pd.read_parquet("data.parquet")

Comencemos comparando el tiempo de calcular la suma de todos los valores de la columna `RETAIL SALES`

In [ ]:
%timeit df_csv["RETAIL SALES"].sum()

1.29 ms ± 47.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%timeit df_parquet["RETAIL SALES"].sum()

1.19 ms ± 35.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Ahora realicemos una operación un poco mas compleja: agrupar por varias columnas y realizar funciones de agregación sobre otras columnas

In [ ]:
%%timeit
df_csv.groupby(["ITEM TYPE", "SUPPLIER"]).agg({
    "WAREHOUSE SALES": "mean",
    "RETAIL SALES": "sum"
    })

59.1 ms ± 1.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit
df_parquet.groupby(["ITEM TYPE", "SUPPLIER"]).agg({
    "WAREHOUSE SALES": "mean",
    "RETAIL SALES": "sum"
    })

47 ms ± 1.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### **Conclusión**

Si bien, en este ejercicio trabajamos con archivos muy pequeños cuyos tiempos de lectura y procesamiento solo llevaban milisegundos o segundos de ejecución, la diferencia es notable cuando lo traslamos a volúmenes mas grandes donde nos podemos ahorrar minutos, horas, etc. de procesamiento.